In [1]:
import numpy as np
import pylab as plb

import sys
#from asciimatics.screen import Screen

from scipy.stats import linregress
import pandas as pd

import statsmodels.api as sm
import pandas.util.testing as tm

import copy

<ipython-input-1-3b51644333a4>:11: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# Источник

url = "https://raw.githubusercontent.com/ForestTiger-GH/COTRA/main/Stat_wheat/Stat_wheat_alldates516_trend20.5.1_nonfilter.txt"

downloaded_url = pd.read_csv(url,
               sep='\t', 
               error_bad_lines=False,
               decimal=',',
               encoding='gbk', 
               skiprows=1)

In [3]:
#downloaded_url.to_numpy()
a = np.asarray(downloaded_url)

columns_all = np.array(a).T

column_dates = columns_all[0].tolist()
width_all = len(a[0])                                                            # Количество всех столбцов
width_y = 1                                                                      # Количество Y (всегда 1)
width_x = width_all - 1 - width_y                                                # Количество Х

column_dates = columns_all[0]                                                    # Столбец дат
np.array(column_dates)
column_y = columns_all[width_all-1]                                              # Столбец Y
np.array(column_y)

lenght_all = len(column_dates)                                                   # Количество строк (на основе дат)

In [4]:
columns_x_all = np.array(columns_all[1:(len(columns_all)-1)]).tolist()           # Группа столбцов X
np.array(columns_x_all).tolist()

lol = lambda lst, sz: [lst[i:i+sz] for i in range(0, len(lst), sz)]
columns_x_all = lol(columns_x_all, width_x)
columns_x_all = columns_x_all[0]

columns_x_all_old = copy.deepcopy(columns_x_all)

In [5]:
check_rate = 0.1                                                                 # Точность проверки (0.1 или 0.01)
check_rate_amount = round(1/check_rate)                                             # то есть 10 вариаций или 100 
check_rate_list = (np.arange(0, 1, check_rate)).tolist()                         # Список уровней (проверка будет на каждом)
check_rate_count = (np.arange(0, check_rate_amount, 1)).tolist()

In [6]:
def reg_m(yyy, xxx):
    ones = np.ones(len(xxx[0]))
    X = sm.add_constant(np.column_stack((xxx[0], ones)))
    for ele in xxx[1:]:
        X = sm.add_constant(np.column_stack((ele, X)))
    results = sm.OLS(yyy, X).fit()
    return results

In [7]:
# Brute-force

list_Rsquare = []
list_Rsquare_all = []
list_Rsquare_x1 = []
list_Rsquare_x2 = []
list_Rsquare_x3 = []
list_Rsquare_x4 = []
list_Rsquare_x5 = []

column_x1 = columns_x_all[0]
column_x2 = columns_x_all[1]
column_x3 = columns_x_all[2]
column_x4 = columns_x_all[3]
column_x5 = columns_x_all[4]

# ------------ Расчет всех шагов и бордеров ------------ #
x1_step_value = max(np.abs(column_x1).tolist())/check_rate_amount
x2_step_value = max(np.abs(column_x2).tolist())/check_rate_amount
x3_step_value = max(np.abs(column_x3).tolist())/check_rate_amount
x4_step_value = max(np.abs(column_x4).tolist())/check_rate_amount
x5_step_value = max(np.abs(column_x5).tolist())/check_rate_amount
border_x1 = []
border_x2 = []
border_x3 = []
border_x4 = []
border_x5 = []
for q in range(check_rate_amount):
    border_x1.append(check_rate_list[q]*x1_step_value*check_rate_amount)
for w in range(check_rate_amount):
    border_x2.append(check_rate_list[w]*x2_step_value*check_rate_amount)
for r in range(check_rate_amount):
    border_x3.append(check_rate_list[r]*x3_step_value*check_rate_amount)
for u in range(check_rate_amount):
    border_x4.append(check_rate_list[u]*x4_step_value*check_rate_amount)
for z in range(check_rate_amount):
    border_x5.append(check_rate_list[z]*x5_step_value*check_rate_amount)

# ------------ Начало цикла ------------ #
for a in range(check_rate_amount):
    for b in range(lenght_all):
        if abs(column_x1[b]) < border_x1[a]:
            column_x1[b] = 0
        
    for c in range(check_rate_amount):
        for d in range(lenght_all):
            if abs(column_x2[d]) < border_x2[c]:
                column_x2[d] = 0
                
        for e in range(check_rate_amount):
            for f in range(lenght_all):
                if abs(column_x3[f]) < border_x3[e]:
                    column_x3[f] = 0

            for g in range(check_rate_amount):
                for h in range(lenght_all):
                    if abs(column_x4[h]) < border_x4[g]:
                        column_x4[h] = 0

                for i in range(check_rate_amount):
                    for j in range(lenght_all):
                        if abs(column_x5[j]) < border_x5[i]:
                            column_x5[j] = 0

                    # Отсюда съезжает вправо при добавлении переменных
                    sys.stdout.write("\rProgress: {} %       Decryption: {} {} {} {} {}     ".format(
                          str(round((int("".join([str(l) for l in [a, c, e, g, i]]))/check_rate_amount**width_x)*100, 1)),
                          str(100*a/float(check_rate_amount))[:1],
                          str(100*c/float(check_rate_amount))[:1],
                          str(100*e/float(check_rate_amount))[:1],
                          str(100*g/float(check_rate_amount))[:1],
                          str(100*i/float(check_rate_amount))[:1]
                          ))
                        
                    columns_x_all[0] = column_x1
                    columns_x_all[1] = column_x2
                    columns_x_all[2] = column_x3
                    columns_x_all[3] = column_x4
                    columns_x_all[4] = column_x5
                    yyy = column_y
                    xxx = columns_x_all
                    yyy = np.array(yyy,'float32')
                    xxx = np.array(xxx,'float32')
                    list_Rsquare_x5.append(reg_m(yyy, xxx).rsquared)
                    list_Rsquare_all.append(reg_m(yyy, xxx).rsquared)

                list_Rsquare_x4.append(np.array(list_Rsquare_x5))

            list_Rsquare_x3.append(np.array(list_Rsquare_x4))

        list_Rsquare_x2.append(np.array(list_Rsquare_x3))

    list_Rsquare_x1.append(np.array(list_Rsquare_x2))

list_Rsquare.append(np.array(list_Rsquare_x1))

# print(len(list_Rsquare[0]))           # x1
# print(len(list_Rsquare[0][0]))        # x2
# print(len(list_Rsquare[0][0][0]))     # x3

Progress: 0.1 %       Decryption: 0 0 1 0 1     

<ipython-input-7-093e2908e5f3>:89: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  list_Rsquare_x3.append(np.array(list_Rsquare_x4))


Progress: 1.0 %       Decryption: 0 0 9 9 9     

<ipython-input-7-093e2908e5f3>:91: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  list_Rsquare_x2.append(np.array(list_Rsquare_x3))


Progress: 10.0 %       Decryption: 1 0 0 1 3     

<ipython-input-7-093e2908e5f3>:93: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  list_Rsquare_x1.append(np.array(list_Rsquare_x2))


Progress: 100.0 %       Decryption: 9 9 9 9 9     

<ipython-input-7-093e2908e5f3>:95: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  list_Rsquare.append(np.array(list_Rsquare_x1))


In [8]:
# ------------ Поиск номера лучшего R-квадрат по общему списку ------------ #
if abs(max(list_Rsquare_all)) >= abs(min(list_Rsquare_all)):
    best_Rsquare = max(list_Rsquare_all)
else:
    best_Rsquare = min(list_Rsquare_all)
best_position_number = list_Rsquare_all.index(best_Rsquare)


# ------------ Расшифровка номера лучшего R-квадрат ------------ #

list_best_positions = list(str(best_position_number).zfill(width_x))      # может быть косяк при вводе чек рейта 0.01
list_best_positions = [int(item) for item in list_best_positions]

best_position_number_x1 = list_best_positions[0]
best_position_number_x2 = list_best_positions[1]
best_position_number_x3 = list_best_positions[2]
best_position_number_x4 = list_best_positions[3]
best_position_number_x5 = list_best_positions[4]

# ------------ Формирование нового массива Х-ов ------------ #
column_x1_new = columns_x_all_old[0]
column_x2_new = columns_x_all_old[1]
column_x3_new = columns_x_all_old[2]
column_x4_new = columns_x_all_old[3]
column_x5_new = columns_x_all_old[4]

for a in range(lenght_all):
    if abs(column_x1_new[a]) < border_x1[best_position_number_x1]:
        column_x1_new[a] = 0
for b in range(lenght_all):
    if abs(column_x2_new[b]) < border_x2[best_position_number_x2]:
        column_x2_new[b] = 0
for c in range(lenght_all):
    if abs(column_x3_new[c]) < border_x3[best_position_number_x3]:
        column_x3_new[c] = 0
for d in range(lenght_all):
    if abs(column_x4_new[d]) < border_x4[best_position_number_x4]:
        column_x4_new[d] = 0
for e in range(lenght_all):
    if abs(column_x5_new[e]) < border_x5[best_position_number_x5]:
        column_x5_new[e] = 0

group_x = []
group_x.append(column_x1_new)
group_x.append(column_x2_new)
group_x.append(column_x3_new)
group_x.append(column_x4_new)
group_x.append(column_x5_new)

In [9]:
print("Результат оптимизации статистики")
print("--------------------------------", end="\n\n")
print("Лучшие уровни отсечки (сколько нужно срезать; в процентах на уровень Check_rate (10 или 100))", end="\n")
print("Было:    [0, 0, 0, 0, 0]")
print("Стало:   {}".format(
    str(list_best_positions)
    ), end="\n\n")

print("Оптимизация R-квадрат:", end="\n")
print("Было:    {} %".format(
    str(round(reg_m(np.array(column_y,'float32'), np.array(columns_x_all_old,'float32')).rsquared*100, 2))
    ))
print("Стало:   {} %".format(
    str(round(reg_m(np.array(column_y,'float32'), np.array(group_x,'float32')).rsquared*100, 2))
    ))

Результат оптимизации статистики
--------------------------------

Лучшие уровни отсечки (сколько нужно срезать; в процентах на уровень Check_rate (10 или 100))
Было:    [0, 0, 0, 0, 0]
Стало:   [0, 0, 0, 0, 0]

Оптимизация R-квадрат:
Было:    7.05 %
Стало:   7.05 %


In [10]:
# Сводка всех результатов регрессии До и После
print(reg_m(np.array(column_y,'float32'), np.array(columns_x_all_old,'float32')).summary())
print('█████████████████████████████████████████████████████████████')
print(reg_m(np.array(column_y,'float32'), np.array(group_x,'float32')).summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.070
Model:                            OLS   Adj. R-squared:                  0.061
Method:                 Least Squares   F-statistic:                     7.733
Date:                Fri, 12 Mar 2021   Prob (F-statistic):           5.03e-07
Time:                        00:02:16   Log-Likelihood:                -1835.9
No. Observations:                 516   AIC:                             3684.
Df Residuals:                     510   BIC:                             3709.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -2.9710      0.950     -3.126      0.0

In [11]:
# Матрица результатов регрессии: R-квадрат (очень большой лист)

#print(list_Rsquare)

In [12]:
columns = []
#columns.append("Dates")        ### пока без дат
for b in range(width_x):
    alfa = "Variable {}".format(str(b+1))
    columns.append(alfa)
    alfa = []
columns.append("Dependent Variable 1")
columns

['Variable 1',
 'Variable 2',
 'Variable 3',
 'Variable 4',
 'Variable 5',
 'Dependent Variable 1']

In [13]:
beta = np.array(group_x).T
merged_one = []
merged_all = []

#np.array(column_dates)

for g in range(len(beta)):
    merged_one = []
    merged_one.append(beta[g].tolist())
    merged_one.append(column_y[g])
    np.array(merged_one)
    merged_all.append(merged_one)

<ipython-input-13-e4b40973e982>:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np.array(merged_one)


In [14]:
csv_half_ready = []
csv_ready = []

for k in range(len(merged_all)):
      for f in range(len(merged_all[0][0])):
          csv_half_ready.append(merged_all[k][0][f])
      csv_half_ready.append(merged_all[k][1])
      np.array(csv_half_ready)
      csv_ready.append(csv_half_ready)
      csv_half_ready = []

In [16]:
data = csv_ready
df = pd.DataFrame(data, columns=columns)
df.to_csv(index=False)

'Variable 1,Variable 2,Variable 3,Variable 4,Variable 5,Dependent Variable 1\r\n0.44535301711,0.26238367523,0.8063665573000001,-0.19073270902,-0.62652733355,6\r\n0.42842250600000004,0.1206220324,0.81383948464,-0.38241413047,-0.59903866142,10\r\n0.41353204844,0.06464619129,0.81278251257,-0.5529158121600001,-0.56899300436,10\r\n0.29274379447,0.02269063352,0.7826692344499999,-0.68367449221,-0.51681387762,12\r\n0.23455245241,0.0007396767999999999,0.76738755696,-0.7808848916600001,-0.45827153554,7\r\n0.1886079902,0.01208829925,0.75441965201,-0.8620520320399999,-0.40910444181,7\r\n0.13215447087999999,0.04105833605,0.72291827898,-0.9268560541099999,-0.36118292982,3\r\n0.08238055307999999,0.08503720504000001,0.68291648782,-0.9415840667,-0.34453432829999997,8\r\n0.047958684620000004,0.07342389621,0.6628557154,-0.92158209392,-0.34715109840999997,13\r\n0.05178582616999999,0.019634052659999998,0.60301842858,-0.8899685007700001,-0.31112126898000003,5\r\n0.12201714582,0.012453145520000002,0.56851471